In [17]:
import pandas as pd
import os
from datetime import date, timedelta
from sqlalchemy import create_engine, text

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = os.path.join(os.path.expanduser("~"), "iCloudDrive")
one_path = os.path.join(os.path.expanduser("~"), "OneDrive","Documents","Data")
osd_path = os.path.join(os.path.expanduser("~"),"OneDrive","Documents","obsidian-git-sync","Data")
dts_path = os.path.join(os.path.expanduser("~"),"Downloads","Datasets")

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}
pd.set_option('display.float_format','{:,.2f}'.format)
year = 2024
refund = 668_000   # 66,490.89 Windfall = 200,000 March
                    # LTF = 181498.8 + 3396.97 + 50380.22 = 235,000 June
                    # 501_000 + 167_000 = 668_000
dividend = 361_000
expense  = 724_000

today = date.today()
today

datetime.date(2025, 1, 2)

### Restart and Run All Cells

### Record selection for transactions

In [7]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % year
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit AS net, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2024
ORDER BY sells.date DESC, name


In [9]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,buy_year,buy_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,net,kind
0,MCS,2024,9,2023,5,8.05,7.60,0.45,"3,000","24,150.00","22,800.00","1,350.00",5.92,"1,246.01",DTD
1,3BBIF,2024,9,2022,7,6.50,9.15,-2.65,"10,000","65,000.00","91,500.00","-26,500.00",-28.96,"-26,846.64",HD
2,GVREIT,2024,9,2024,2,6.95,6.00,0.95,"6,000","41,700.00","36,000.00","5,700.00",15.83,"5,527.90",HD
3,IVL,2024,9,2024,6,23.90,20.00,3.90,800,"19,120.00","16,000.00","3,120.00",19.50,"3,042.21",HD
4,CPNREIT,2024,9,2023,3,12.70,12.60,0.10,"5,000","63,500.00","63,000.00",500.00,0.79,219.81,HD
5,ORI,2024,9,2024,7,5.60,4.60,1.00,"3,000","16,800.00","13,800.00","3,000.00",21.74,"2,932.22",HD
6,JMT,2024,8,2024,6,16.40,13.00,3.40,"1,800","29,520.00","23,400.00","6,120.00",26.15,"6,002.78",DTD
7,WHART,2024,8,2024,5,10.30,8.80,1.50,"5,000","51,500.00","44,000.00","7,500.00",17.05,"7,288.47",HD
8,JMT,2024,8,2024,7,14.90,11.50,3.40,"1,500","22,350.00","17,250.00","5,100.00",29.57,"5,012.29",HD
9,WHART,2024,8,2024,6,10.10,8.60,1.50,"5,000","50,500.00","43,000.00","7,500.00",17.44,"7,292.90",HD


In [11]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum()

sell_year  sell_month  buy_year  buy_month  name   
2024       1           2021      9          BCH         2,202.42
                       2023      9          STA         2,067.82
                                 10         TFFIF         924.68
                                 12         TFFIF         802.74
                       2024      1          CRC            56.65
           2           2024      1          BEM            49.99
           3           2024      2          DIF          -255.32
           6           2024      6          JMT         1,146.37
           8           2024      5          WHART       7,288.47
                                 6          JMT         6,002.78
                                            WHART       7,292.90
                                 7          JMT         5,012.29
           9           2022      7          3BBIF     -26,846.64
                       2023      3          CPNREIT       219.81
                                 5    

In [13]:
sells_df.groupby(['name'])[['gross','net']].sum().head(5)

,gross,net
name,,
3BBIF,"-26,500.00","-26,846.64"
BCH,"2,400.00","2,202.42"
BEM,150.00,49.99
CPNREIT,500.00,219.81
CRC,150.00,56.65


In [6]:
total_df = sells_df.groupby(['name'])[['gross','net']].sum()
total_df.nlargest(5, 'gross')[['gross','net']].style.format(format_dict)

,gross,net
name,,
WHART,"15,000.00","14,581.37"
JMT,"12,480.00","12,161.44"
GVREIT,"5,700.00","5,527.90"
IVL,"3,120.00","3,042.21"
ORI,"3,000.00","2,932.22"


In [19]:
file_name = 'ytd-profit.csv'
data_file = data_path + file_name
csv_file = csv_path + file_name
osd_file = osd_path + file_name

sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(csv_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(data_file)
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).net.sum().to_csv(osd_file)

In [21]:
output = sells_df.groupby(['sell_month']).net.sum()
output

sell_month
1     6,054.31
2        49.99
3      -255.32
6     1,146.37
8    25,596.44
9   -13,878.49
Name: net, dtype: float64

In [23]:
#data_path = "../data/"
file_name = 'tmp-file-of-profits.csv'
data_file = data_path + file_name
osd_file = osd_path + file_name

output.to_csv(data_file, index=True)
output.to_csv(osd_file, index=True)

In [25]:
profit = sells_df.net.sum()
ttl_profit = profit + dividend
profit,dividend,ttl_profit

(18713.3, 361000, 379713.3)

In [27]:
profit = sells_df.net.sum()
net = profit + dividend - expense
net = round(net / 1000,0) * 1000
round(profit,0),dividend,expense, round(net,0)

(18713.0, 361000, 724000, -344000.0)

### Bottom line of up to date transactions

In [30]:
trade = round(profit,0) + dividend
trade = round(trade / 1000,0) * 1000
trade

380000.0

In [32]:
total = refund + net
refund, round(net,0), round(total,0)

(668000, -344000.0, 324000.0)